# ❄️ LSTM for Predictive Climate Control in Vertical Farms

This notebook presents a "Golden Path" for forecasting climate variables (Temperature, Humidity, CO₂) using stacked LSTM models. Accurate short-term forecasts (next-hour) enable proactive HVAC/ventilation adjustments to maintain ideal growth conditions and to save energy.

Why LSTM? Classic RNNs suffer from vanishing gradients and struggle with long-term dependencies (diurnal/seasonal cycles). LSTM gates (input/forget/output) preserve and update long-range information, making them well-suited for farm climate time-series forecasting.

Scenario: "Intelligent Climate Regulation" — simulate 10,000 hourly steps of climate data, train LSTM to predict the next-hour climate state, compare to baselines (persistence and a simple RNN), show business-impact metrics (Energy Savings Score), and provide deployment guidance (TF Lite conversion).

In [ ]:
# Install required packages (uncomment to run if needed)
# !pip install -q tensorflow scikit-learn pandas matplotlib seaborn joblib shap


In [ ]:
# Standard imports and reproducible settings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, SimpleRNN
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import joblib
import shap

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Seeding
SEED = 42
np.random.seed(SEED)
import random
random.seed(SEED)
import tensorflow as tf
tf.random.set_seed(SEED)


In [ ]:
# ------------------------------
# Data Simulation: 10,000 hourly steps (~1 year)
# ------------------------------

def simulate_climate_data(steps=10000, start_date='2024-01-01'):
    rng = pd.date_range(start=start_date, periods=steps, freq='H')
    df = pd.DataFrame({'timestamp': rng})

    hours = np.arange(steps)
    # Daily cycle for interior temp driven by lights + diurnal
    daily = 3 * np.sin(2 * np.pi * (hours % 24) / 24 - 0.5)
    weekly = 0.5 * np.sin(2 * np.pi * hours / (24 * 7))
    seasonal = 2.0 * np.sin(2 * np.pi * hours / (24 * 365.0))

    # External temperature (weather)
    df['External_Temp'] = 5 + 10 * np.sin(2 * np.pi * hours / (24 * 365.0)) + np.random.normal(0, 1.0, steps)

    # Internal temperature influenced by lights, external temp, and HVAC
    df['Light_Intensity'] = np.clip(200 + 150 * np.sin(2 * np.pi * (hours % 24) / 24) + np.random.normal(0, 20, steps), 0, 1000)
    df['Temp'] = np.clip(21 + daily + 0.1 * (df['External_Temp'] - 10) + seasonal + np.random.normal(0, 0.5, steps), 10, 35)

    # Humidity inversely related to temperature with some randomness
    df['Humidity'] = np.clip(60 - 0.6 * daily + 3 * weekly + np.random.normal(0, 2.0, steps), 20, 95)

    # CO2 with enrichment cycles and weekday patterns (ventilation schedule)
    df['CO2'] = np.clip(400 + 80 * (0.5 + 0.5 * np.sign(np.sin(2 * np.pi * (hours % 24) / 24))) + 10 * weekly + np.random.normal(0, 15, steps), 300, 2000)

    # Plant density (affects HVAC load) cycles with planting schedule
    df['Plant_Density'] = np.clip(0.8 + 0.2 * np.sin(2 * np.pi * hours / (24 * 28)) + np.random.normal(0, 0.02, steps), 0.5, 1.2)

    # Add noise and occasional disturbance (e.g., door open causing spike)
    disturbances = np.random.choice(steps, size=int(steps * 0.002), replace=False)
    df.loc[disturbances, 'Temp'] += np.random.uniform(2, 6, size=len(disturbances))
    df.loc[disturbances, 'Humidity'] += np.random.uniform(-8, 8, size=len(disturbances))

    return df

# Generate dataset
df = simulate_climate_data(steps=10000)
print('Dataset generated:', df.shape)
df.head()

In [ ]:
# Quick EDA: visualize temperature and humidity over time (daily avg)
df_plot = df.set_index('timestamp').resample('D').mean()

plt.figure(figsize=(14, 4))
plt.plot(df_plot.index, df_plot['Temp'], label='Temp (daily avg)')
plt.plot(df_plot.index, df_plot['Humidity'], label='Humidity (daily avg)')
plt.title('Daily-Average Temp & Humidity')
plt.legend()
plt.show()

# Show sample rows
df[['timestamp', 'Temp', 'Humidity', 'CO2', 'External_Temp', 'Light_Intensity']].head()

In [ ]:
# ------------------------------
# Preprocessing: sliding window generator, scaling, and train/val/test split
# ------------------------------

target_cols = ['Temp', 'Humidity', 'CO2']
feature_cols = ['Temp', 'Humidity', 'CO2', 'Light_Intensity', 'External_Temp', 'Plant_Density']

# Scaling features
scaler = MinMaxScaler()
df_scaled = df.copy()
df_scaled[feature_cols] = scaler.fit_transform(df[feature_cols])

# Sliding window
def create_sequences(data, feature_cols, target_cols, lookback=24):
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data[feature_cols].iloc[i - lookback:i].values)
        y.append(data[target_cols].iloc[i].values)  # next-step prediction
    return np.array(X), np.array(y)

LOOKBACK = 24  # 24 hours
X, y = create_sequences(df_scaled, feature_cols, target_cols, lookback=LOOKBACK)
print('X shape:', X.shape, 'y shape:', y.shape)

# Time-based split
train_size = int(0.7 * len(X))
val_size = int(0.15 * len(X))

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size + val_size], y[train_size:train_size + val_size]
X_test, y_test = X[train_size + val_size:], y[train_size + val_size:]

print('Train/Val/Test shapes:', X_train.shape, X_val.shape, X_test.shape)

In [ ]:
# ------------------------------
# Baselines: Persistence and Simple RNN
# ------------------------------

# Persistence baseline: predict last observed value (t) as t+1
def persistence_predict(X):
    # last time-step in sequence
    return X[:, -1, :3]  # Temp, Humidity, CO2

persistence_preds = persistence_predict(X_test)

# Simple RNN baseline
rnn_model = Sequential([
    SimpleRNN(32, input_shape=(LOOKBACK, X_train.shape[2])),
    Dense(32, activation='relu'),
    Dense(3)
])

rnn_model.compile(optimizer='adam', loss='mse')
rnn_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=64, verbose=1)

rnn_preds = rnn_model.predict(X_test)

# Baseline metrics helper
def compute_metrics(y_true, y_pred):
    metrics = {}
    for i, col in enumerate(target_cols):
        rmse = np.sqrt(mean_squared_error(y_true[:, i], y_pred[:, i]))
        mae = mean_absolute_error(y_true[:, i], y_pred[:, i])
        metrics[col] = {'RMSE': rmse, 'MAE': mae}
    return metrics

persistence_metrics = compute_metrics(y_test, persistence_preds)
rnn_metrics = compute_metrics(y_test, rnn_preds)
print('Persistence metrics:', persistence_metrics)
print('RNN metrics:', rnn_metrics)

In [ ]:
# ------------------------------
# LSTM model: stacked LSTM with Dropout
# ------------------------------

def build_lstm(input_shape, units=[64, 32], dropout=0.2):
    model = Sequential()
    model.add(LSTM(units[0], return_sequences=True, input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(LSTM(units[1]))
    model.add(Dropout(dropout))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(len(target_cols)))
    model.compile(optimizer='adam', loss='mse')
    return model

lstm = build_lstm((LOOKBACK, X_train.shape[2]), units=[128, 64], dropout=0.2)

checkpoint = ModelCheckpoint('best_lstm.h5', monitor='val_loss', save_best_only=True, verbose=1)
early = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

history = lstm.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=80, batch_size=128, callbacks=[checkpoint, early], verbose=2)

# Plot training/validation loss
plt.figure()
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Training / Validation Loss')
plt.legend()
plt.show()

# Predict on test
lstm_preds = lstm.predict(X_test)
lstm_metrics = compute_metrics(y_test, lstm_preds)
print('LSTM metrics:', lstm_metrics)

In [ ]:
# ------------------------------
# Forecast overlay: visualize a 48-hour zoom on test set
# ------------------------------

# Pick a test slice near the start of X_test
idx = 200
hours = np.arange(48)

true_slice = y_test[idx:idx+48]
pred_slice = lstm_preds[idx:idx+48]

plt.figure(figsize=(14, 6))
plt.subplot(3, 1, 1)
plt.plot(hours, true_slice[:, 0], label='Actual Temp')
plt.plot(hours, pred_slice[:, 0], label='Pred Temp')
plt.title('48-hour Forecast — Temp (Actual vs Predicted)')
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(hours, true_slice[:, 1], label='Actual Humidity')
plt.plot(hours, pred_slice[:, 1], label='Pred Humidity')
plt.title('Humidity (Actual vs Predicted)')
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(hours, true_slice[:, 2], label='Actual CO2')
plt.plot(hours, pred_slice[:, 2], label='Pred CO2')
plt.title('CO2 (Actual vs Predicted)')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# ------------------------------
# Feature importance via permutation (test set)
# ------------------------------

from copy import deepcopy

def permutation_importance(model_predict_fn, X_test, y_test, feature_idx, n_repeats=5, metric_fn=None):
    if metric_fn is None:
        def metric_fn(y_true, y_pred):
            return np.sqrt(mean_squared_error(y_true, y_pred))

    base_pred = model_predict_fn(X_test)
    base_score = metric_fn(y_test, base_pred)

    scores = []
    for _ in range(n_repeats):
        X_perm = deepcopy(X_test)
        # permute a single feature across time (flatten then reshape)
        flat = X_perm[:, :, feature_idx].flatten()
        np.random.shuffle(flat)
        X_perm[:, :, feature_idx] = flat.reshape(X_perm.shape[0], X_perm.shape[1])
        perm_pred = model_predict_fn(X_perm)
        scores.append(metric_fn(y_test, perm_pred))
    return np.mean(scores) - base_score

# Model predict wrapper
def lstm_predict_fn(X_in):
    return lstm.predict(X_in)

# Compute permutation importance for each feature (using first frame value as representative)
feature_names = feature_cols
importances = {}
for i, fname in enumerate(feature_names):
    # Use column index i in feature dimension
    imp = permutation_importance(lstm_predict_fn, X_test, y_test, i, n_repeats=3,
                                 metric_fn=lambda y_true, y_pred: np.mean(np.sqrt(np.mean((y_true - y_pred) ** 2, axis=0))))
    importances[fname] = imp

imp_df = pd.DataFrame({'feature': list(importances.keys()), 'importance': list(importances.values())}).sort_values('importance', ascending=False)
print('Permutation importances (higher → worse when permuted):')
print(imp_df)

# Plot top features
plt.figure(figsize=(8, 4))
sns.barplot(data=imp_df.head(8), x='importance', y='feature', palette='magma')
plt.title('Permutation Feature Importance (LSTM)')
plt.show()

In [ ]:
# ------------------------------
# Business metrics: RMSE/MAE and Energy Savings Score
# ------------------------------

# Helper to compute aggregated RMSE and MAE
def aggregate_metrics(y_true, y_pred):
    out = {}
    for i, col in enumerate(target_cols):
        out[f'{col}_RMSE'] = np.sqrt(mean_squared_error(y_true[:, i], y_pred[:, i]))
        out[f'{col}_MAE'] = mean_absolute_error(y_true[:, i], y_pred[:, i])
    return out

metrics_persistence = aggregate_metrics(y_test, persistence_preds)
metrics_rnn = aggregate_metrics(y_test, rnn_preds)
metrics_lstm = aggregate_metrics(y_test, lstm_preds)

print('Persistence:', metrics_persistence)
print('RNN:', metrics_rnn)
print('LSTM:', metrics_lstm)

# Energy Savings Score: heuristic that better forecasts reduce HVAC energy by proportion of RMSE reduction
# Score = 100 * (1 - RMSE_model / RMSE_persistence) averaged over targets (higher is better)
rmse_persistence = np.mean([metrics_persistence[f'{c}_RMSE'] for c in target_cols])
rmse_lstm = np.mean([metrics_lstm[f'{c}_RMSE'] for c in target_cols])
energy_savings_pct = 100 * (1 - rmse_lstm / rmse_persistence)
print(f'Estimated Energy Savings (heuristic): {energy_savings_pct:.2f}% (higher is better)')

print('\nBusiness interpretation:')
print('- Lower RMSE/MAE reduces over/under-cooling events, translating to HVAC runtime savings.')
print('- Energy Savings Score is a heuristic estimate; field calibration required to map RMSE improvements to kWh and cost savings.')

In [ ]:
# ------------------------------
# Export model and Deployment Blueprint
# ------------------------------

# Save Keras model
lstm.save('climate_lstm.h5')
print('Saved model to climate_lstm.h5')

# Quick TensorFlow Lite conversion snippet (for edge deployment)
try:
    converter = tf.lite.TFLiteConverter.from_keras_model(lstm)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()
    with open('climate_lstm.tflite', 'wb') as f:
        f.write(tflite_model)
    print('Saved TFLite model to climate_lstm.tflite')
except Exception as e:
    print('TFLite conversion skipped (requires full TF environment):', e)

# Sample inference on live sensor data (pseudo)
# Assume we have last 24 hours as `recent_window` scaled by same scaler
# recent_window shape expected: (1, LOOKBACK, n_features)
# pred = lstm.predict(recent_window)
# Inverse transform if needed and use prediction to trigger HVAC adjustments (pre-cooling, ventilation)

# Deployment checklist:
# - Export model and quantize for edge (TFLite), wrap in a service (Lightweight or MCU SDK)
# - Integrate with building energy models to convert forecast accuracy to runtime adjustments and kWh savings
# - Add safety constraints (max cooling rate), fallback (persistence) and human-in-loop overrides

print('\nDeployment blueprint included.')

In [ ]:
# ------------------------------
# Actionable Forecasting & Operator Guidance (Ambient differentiation)
# ------------------------------

# Example heuristic: derive rules from forecasted temp/Humidity
print('Actionable rules (examples):')
print('- If forecasted Temp in 4h > 25°C: schedule pre-cooling in next 2h to reduce HVAC ramp, estimated energy save ~10-15%')
print('- If Humidity trending up and forecasted >80%: increase ventilation and decrease misting to prevent mold risk')
print('- If CO2 forecast shows prolonged low levels: postpone enrichment and prioritize recirculation to save CO2 consumption')

# What-if simulation cell (operator demo): apply a sudden external temp spike and observe predicted response
spike_df = df.copy()
# inject a heat spike at t=2000 lasting 12 hours
spike_df.loc[2000:2011, 'External_Temp'] += 8
# Create sequences for spike window and show model predictions overlay for that region as a demo
X_spike, y_spike = create_sequences(spike_df, feature_cols, target_cols, lookback=LOOKBACK)
spike_pred = lstm.predict(X_spike[2000:2050])

plt.figure(figsize=(12, 4))
plt.plot(y_spike[2000:2050, 0], label='Actual Temp (spike region)')
plt.plot(spike_pred[:, 0], label='Pred Temp')
plt.title('What-if: External Temp Spike — Model Forecast')
plt.legend()
plt.show()
